# Gotcha's from Pandas to Dask

This notebook highlights some key differences when transfering code from `Pandas` to run in a `Dask` environment.  
Most issues have a link to the [Dask documentation](https://docs.dask.org/en/latest/) for additional information.

In [1]:
# since Dask is activly beeing developed - the current example is running with the below version
import dask
import dask.dataframe as dd
import pandas as pd
print(f'Dask versoin: {dask.__version__}')
print(f'Pandas versoin: {pd.__version__}')

Dask versoin: 2022.05.0
Pandas versoin: 1.4.2


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  In this example we are running on a `LocalCluster`, this  will also provide a dashboard which is useful to gain insight on the computation.  
For additional information on [Dask Client see documentation](https://docs.dask.org/en/latest/setup.html?highlight=client#setup)  

The link to the dashboard will become visible when you create a client (as shown below).  
When running within `Jupyter Lab` an [extenstion](https://github.com/dask/dask-labextension) can be installed to view the various dashboard widgets. 

In [2]:
from dask.distributed import Client
# client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 2
Total threads: 2,Total memory: 6.78 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44205,Workers: 2
Dashboard: http://127.0.0.1:8787/status,Total threads: 2
Started: Just now,Total memory: 6.78 GiB
Comm: tcp://127.0.0.1:40711,Total threads: 1
Dashboard: http://127.0.0.1:46039/status,Memory: 3.39 GiB
Nanny: tcp://127.0.0.1:38003,


See [documentation for addtional cluster configuration](http://distributed.dask.org/en/latest/local-cluster.html)

# Create 2 DataFrames for comparison: 
1. for Dask 
2. for Pandas  
Dask comes with builtin dataset samples, we will use this sample for our example. 

In [3]:
ddf = dask.datasets.timeseries()
ddf

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


* Remember `Dask framework` is **lazy** thus in order to see the result we need to run [compute()](https://docs.dask.org/en/latest/generated/dask.dataframe.DataFrame.compute.html) 
 (or `head()` which runs under the hood compute()) )

In [4]:
ddf.head(2)

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551


#### Pandas Dataframe
In order to create a `Pandas` dataframe we can use the `compute()` method from a `Dask dataframe`

In [5]:
pdf = ddf.compute()  
print(type(pdf))
pdf.head(2)

<class 'pandas.core.frame.DataFrame'>


,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551


### dataframe.shape  
We can also see *dask laziness* when using the shape attribute

In [6]:
print(f'Pandas shape: {pdf.shape}')
print('---------------------------')
print(f'Dask lazy shape: {ddf.shape}')

Pandas shape: (2592000, 4)
---------------------------
Dask lazy shape: (Delayed('int-573bbb04-4b7b-41dc-bdd2-29b45fe82e6d'), 4)


We cannot get the full shape before accessing all the partitions - running `len` will do so

In [7]:
print(f'Dask computed shape: {len(ddf.index):,}')  # expensive

Dask computed shape: 2,592,000


## Creating a `Dask dataframe` from `Pandas`
In order to utilize `Dask` capablities on an existing `Pandas dataframe` (pdf) we need to convert the `Pandas dataframe` into a `Dask dataframe` (ddf)  with the [from_pandas](https://docs.dask.org/en/latest/generated/dask.dataframe.from_pandas.html) method. 
You must supply the number of partitions or chunksize that will be used to generate the dask dataframe

In [8]:
ddf2 = dask.dataframe.from_pandas(pdf, npartitions=10)
ddf2

,id,name,x,y
npartitions=10,,,,
2000-01-01 00:00:00,int64,object,float64,float64
2000-01-04 00:00:00,...,...,...,...
...,...,...,...,...
2000-01-28 00:00:00,...,...,...,...
2000-01-30 23:59:59,...,...,...,...


## Partitions in Dask Dataframes

Notice that when we created a `Dask dataframe` we needed to supply an argument of `npartitions`.  
    The number of partitions will assist `Dask` on how to breakup the  `Pandas Datafram` and parallelize the computation.  
Each partition is a *separate* dataframe. For additional information see [partition documentation](https://docs.dask.org/en/latest/dataframe-design.html?highlight=meta%20utils#partitions)  

An example for this can be seen when examing the `reset_ index()` method:

In [9]:
pdf2 = pdf.reset_index()
# Only 1 row
pdf2.loc[0]

timestamp    2000-01-01 00:00:00
id                           970
name                      George
x                       0.367233
y                       0.240946
Name: 0, dtype: object

In [10]:
ddf2 = ddf2.reset_index()
# each partition has an index=0
ddf2.loc[0].compute() 

,timestamp,id,name,x,y
0,2000-01-01,970,George,0.367233,0.240946
0,2000-01-04,930,George,-0.866855,0.441477
0,2000-01-07,1060,Ingrid,-0.731187,0.597793
0,2000-01-10,1047,Victor,-0.527911,-0.318880
0,2000-01-13,1034,Edith,0.957465,0.991644
0,2000-01-16,1028,Patricia,-0.018618,0.357750
0,2000-01-19,1009,Victor,0.722396,-0.601105
0,2000-01-22,992,Michael,0.197673,0.011724
0,2000-01-25,963,Dan,0.249601,-0.119814
0,2000-01-28,961,Victor,-0.281209,0.917524


# Dask Dataframe vs Pandas Dataframe
Now that we have a `dask` (ddf) and a `pandas` (pdf) dataframe we can start to compair the interactions with them.

## Conceptual shift - from Update to Insert/Delete
Dask does not update - thus there are no arguments such as `inplace=True` which exist in Pandas.  
For more detials see [issue#653 on github](https://github.com/dask/dask/issues/653)

### Rename Columns

* using `inplace=True` is not considerd to be *best practice*. 

In [11]:
# Pandas 
print(pdf.columns)
# pdf.rename(columns={'id':'ID'}, inplace=True)
pdf = pdf.rename(columns={'id':'ID'})
pdf.columns

Index(['id', 'name', 'x', 'y'], dtype='object')


Index(['ID', 'name', 'x', 'y'], dtype='object')

In [12]:
# Dask
print(ddf.columns)
ddf = ddf.rename(columns={'id':'ID'})
ddf.columns

Index(['id', 'name', 'x', 'y'], dtype='object')


Index(['ID', 'name', 'x', 'y'], dtype='object')

## Data manipulations  
There are several diffrences when manipulating data.  

### loc - Pandas

In [13]:
cond_pdf = (pdf['x']>0.5) & (pdf['x']<0.8)
pdf.loc[cond_pdf, ['y']] = pdf['y']* 100
pdf[cond_pdf].head(2)

,ID,name,x,y
timestamp,,,,
2000-01-01 00:00:02,962,Yvonne,0.656793,-26.767312
2000-01-01 00:00:16,940,Yvonne,0.720565,-22.366579


### Error

```python
cond_ddf = (ddf['x']>0.5) & (ddf['x']<0.8)
ddf.loc[cond_ddf, ['y']] = ddf['y']* 100
ddf[cond_ddf].head(2)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Input In [14], in <cell line: 2>()
      1 cond_ddf = (ddf['x']>0.5) & (ddf['x']<0.8)
----> 2 ddf.loc[cond_ddf, ['y']] = ddf['y']* 100
      3 ddf[cond_ddf].head(2)

TypeError: '_LocIndexer' object does not support item assignment
```

### Dask - use mask/where

In [14]:
# Pandas
pdf['y'] = pdf['y'].mask(cond=cond_pdf, other=pdf['y']* 100)
pdf.head(2)

,ID,name,x,y
timestamp,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551


In [15]:
#Dask
cond_ddf = (ddf['x']>0.5) & (ddf['x']<0.8)
ddf['y'] = ddf['y'].mask(cond=cond_ddf, other=ddf['y']* 100)
ddf.head(2)

,ID,name,x,y
timestamp,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551


For more information see [dask mask documentation](https://docs.dask.org/en/latest/generated/dask.dataframe.DataFrame.mask.html)

## Meta argument
One key feature in `Dask` is the introduction of `meta` arguement.  
> `meta` is the prescription of the names/types of the output from the computation  
from [stack overflow answer](https://stackoverflow.com/questions/44432868/dask-dataframe-apply-meta)

Since `Dask` creates a DAG for the computation, it requires to understand what are the outputs of each calculation stage.  
For additinal information see [meta documentation](https://docs.dask.org/en/latest/dataframe-design.html?highlight=meta%20utils#metadata)

In [16]:
pdf['initials'] = pdf['name'].apply(lambda x: x[0]+x[1])
pdf.head(2)

,ID,name,x,y,initials
timestamp,,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946,Ge
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551,Yv


In [17]:
# Dask - Warning
ddf['initials'] = ddf['name'].apply(lambda x: x[0]+x[1])
ddf.head(2)

/usr/share/miniconda3/envs/dask-examples/lib/python3.9/site-packages/dask/dataframe/core.py:3946: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('name', 'object'))

  warnings.warn(meta_warning(meta))


,ID,name,x,y,initials
timestamp,,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946,Ge
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551,Yv


#### Introducing meta argument

In [18]:
# Describe the outcome type of the calculation
meta_arg = pd.Series(object, name='initials')

In [19]:
ddf['initials'] = ddf['name'].apply(lambda x: x[0]+x[1], meta=meta_arg)
ddf.head(2)

,ID,name,x,y,initials
timestamp,,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946,Ge
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551,Yv


In [20]:
# similar when using a function
def func(row):
    if (row['x']> 0):
        return row['x'] * 1000  
    else:
        return row['y'] * -1

In [21]:
ddf['z'] = ddf.apply(func, axis=1, meta=('z', 'float'))
ddf.head(2)

,ID,name,x,y,initials,z
timestamp,,,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946,Ge,367.233154
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551,Yv,56.227101


### Map partitions
* We can supply an ad-hoc function to run on each partition using the [map_partitions](https://docs.dask.org/en/latest/generated/dask.dataframe.DataFrame.map_partitions.html) method.   
Mainly useful for functions that are not implemented in `Dask` or `Pandas` . 
* Finally we can return a new `dataframe` which needs to be described in the `meta` argument  
The function could also include arguments.

In [22]:
import numpy as np
def func2(df, coor_x, coor_y, drop_cols):
    df['dist'] =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2  
                           +  (df[coor_y] - df[coor_y].shift())**2 )
    return df.drop(drop_cols, axis=1)

ddf2 = ddf.map_partitions(func2
                          , coor_x='x'
                          , coor_y='y'
                          , drop_cols=['initials', 'z']
                          , meta=pd.DataFrame({'ID':'i8'
                                              , 'name':str
                                              , 'x':'f8'
                                              , 'y':'f8'                                              
                                              , 'dist':'f8'}, index=[0]))
ddf2.head()

,ID,name,x,y,dist
timestamp,,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946,NaN
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551,0.496173
2000-01-01 00:00:02,962,Yvonne,0.656793,-26.767312,27.401445
2000-01-01 00:00:03,1019,Norbert,-0.132204,0.879199,27.657768
2000-01-01 00:00:04,999,Laura,-0.137095,-0.555277,1.434485


### Convert index into Time column

In [23]:
# Only Pandas
pdf = pdf.assign(times=pd.to_datetime(pdf.index).time)
pdf.head(2)

,ID,name,x,y,initials,times
timestamp,,,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946,Ge,00:00:00
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551,Yv,00:00:01


In [24]:
# Dask or Pandas
ddf = ddf.assign(times=ddf.index.astype('M8[ns]'))
# or  ddf = ddf.assign(Time= dask.dataframe.to_datetime(ddf.index, format='%Y-%m-%d'). )
ddf['times'] = ddf['times'].dt.time
ddf =client.persist(ddf)
ddf.head(2)

,ID,name,x,y,initials,z,times
timestamp,,,,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946,Ge,367.233154,00:00:00
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551,Yv,56.227101,00:00:01


## Drop NA on column

In [25]:
# no issue with regular drop columns
pdf = pdf.drop(labels=['initials'],axis=1)
ddf = ddf.drop(labels=['initials','z'],axis=1) 

In [26]:
# Pandas
pdf = pdf.assign(colna = None)
# Dask
ddf = ddf.assign(colna = None)

In [27]:
pdf = pdf.dropna(axis=1, how='all')
pdf.head(2)

,ID,name,x,y,times
timestamp,,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946,00:00:00
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551,00:00:01


In odrer for `Dask` to drop a column with all `na` it must check all the partitions with `compute()`

In [28]:
if ddf.colna.isnull().all().compute() == True:   # check if all values in column are Null -  expensive
    ddf = ddf.drop(labels=['colna'],axis=1)
ddf.head(2)

,ID,name,x,y,times
timestamp,,,,,
2000-01-01 00:00:00,970,George,0.367233,0.240946,00:00:00
2000-01-01 00:00:01,999,Yvonne,0.056227,0.627551,00:00:01


##  1.4 Reset Index

In [29]:
# Pandas
pdf =pdf.reset_index(drop=True)
pdf.head(2)

,ID,name,x,y,times
0,970,George,0.367233,0.240946,00:00:00
1,999,Yvonne,0.056227,0.627551,00:00:01


In [30]:
# Dask
ddf = ddf.reset_index()
ddf = ddf.drop(labels=['timestamp'], axis=1 )
ddf.head(2)

,ID,name,x,y,times
0,970,George,0.367233,0.240946,00:00:00
1,999,Yvonne,0.056227,0.627551,00:00:01


# Read / Save files

* When working with `pandas` and `dask` preferable use [parquet format](https://docs.dask.org/en/latest/dataframe-best-practices.html?highlight=parquet#store-data-in-apache-parquet-format).  
* When working with `Dask` - files can be read with multiple workers .  
* Most `kwargs` are applicable for reading and writing files   
e.g. 
ddf = dd.read_csv('data/pd2dd/ddf*.csv', compression='gzip', header=False).  
* However some are not available such as  `nrows`.

[see documentaion](https://docs.dask.org/en/latest/generated/dask.dataframe.DataFrame.to_csv.html) (including the option for output file naming).

## Save files

In [31]:
from pathlib import Path
output_dir_file = Path('data/pdf_single_file.csv')
output_dir_file.parent.mkdir(parents=True, exist_ok=True)

In [32]:
%%time
# Pandas
pdf.to_csv(output_dir_file)

CPU times: user 16.2 s, sys: 617 ms, total: 16.8 s
Wall time: 16.3 s


In [33]:
list(output_dir_file.parent.glob('*.csv'))

[PosixPath('data/2000-01-12.csv'),
 PosixPath('data/2000-01-11.csv'),
 PosixPath('data/2000-01-23.csv'),
 PosixPath('data/2000-01-08.csv'),
 PosixPath('data/2000-01-17.csv'),
 PosixPath('data/2000-01-13.csv'),
 PosixPath('data/2000-01-03.csv'),
 PosixPath('data/2000-01-14.csv'),
 PosixPath('data/2000-01-25.csv'),
 PosixPath('data/2000-01-19.csv'),
 PosixPath('data/2000-01-18.csv'),
 PosixPath('data/2000-01-24.csv'),
 PosixPath('data/2000-01-20.csv'),
 PosixPath('data/2000-01-02.csv'),
 PosixPath('data/2000-01-09.csv'),
 PosixPath('data/2000-01-26.csv'),
 PosixPath('data/2000-01-15.csv'),
 PosixPath('data/2000-01-27.csv'),
 PosixPath('data/2000-01-22.csv'),
 PosixPath('data/2000-01-10.csv'),
 PosixPath('data/2000-01-05.csv'),
 PosixPath('data/2000-01-29.csv'),
 PosixPath('data/2000-01-04.csv'),
 PosixPath('data/2000-01-21.csv'),
 PosixPath('data/2000-01-16.csv'),
 PosixPath('data/2000-01-28.csv'),
 PosixPath('data/pdf_single_file.csv'),
 PosixPath('data/2000-01-01.csv'),
 PosixPath('dat

Notice the `'*'` to allow for multiple file renaming. 

In [34]:
output_dask_dir = Path('data/dask_multi_files/')
output_dask_dir.mkdir(parents=True, exist_ok=True)

In [35]:
%%time
# Dask
ddf.to_csv(f'{output_dask_dir}/ddf*.csv', index = False)

CPU times: user 501 ms, sys: 53.1 ms, total: 555 ms
Wall time: 10.6 s


['/home/runner/work/dask-examples/dask-examples/dataframes/data/dask_multi_files/ddf00.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/dask_multi_files/ddf01.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/dask_multi_files/ddf02.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/dask_multi_files/ddf03.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/dask_multi_files/ddf04.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/dask_multi_files/ddf05.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/dask_multi_files/ddf06.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/dask_multi_files/ddf07.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/dask_multi_files/ddf08.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/dask_multi_files/ddf09.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/dask_m

To find the number of partitions which will determine the number of output files use [dask.dataframe.npartitions](https://docs.dask.org/en/latest/generated/dask.dataframe.DataFrame.npartitions.html)  

To change the number of output files use [repartition](https://docs.dask.org/en/latest/generated/dask.dataframe.DataFrame.repartition.html) which is an expensive operation.

In [36]:
ddf.npartitions

30

## Read Multiple files

For `pandas` it is possible to iterate and concat the files [see answer from stack overflow](https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe).

In [37]:
%%time
# Pandas
concat_df = pd.concat([pd.read_csv(f) 
                       for f in list(output_dask_dir.iterdir())])
len(concat_df)

CPU times: user 2.61 s, sys: 370 ms, total: 2.98 s
Wall time: 2.92 s


2592000

In [38]:
%%time
# Dask
_ddf = dd.read_csv(output_dask_dir/'ddf*.csv')
_ddf

CPU times: user 11.6 ms, sys: 0 ns, total: 11.6 ms
Wall time: 10.9 ms


,ID,name,x,y,times
npartitions=30,,,,,
,int64,object,float64,float64,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


Remember that `Dask` is lazy - thus it does not *realy* read the file until it needs to...

In [39]:
%%time
_ddf = dd.read_csv(output_dask_dir/'ddf*.csv')
len(_ddf)

CPU times: user 79.6 ms, sys: 8.3 ms, total: 87.9 ms
Wall time: 864 ms


2592000

 ## Consider using client.persist()
 Since Dask is lazy - it may run the **entire** graph/DAG (again) even if it already run part of the calculation in a previous cell.  Thus use [persist](https://docs.dask.org/en/latest/dataframe-best-practices.html?highlight=parquet#persist-intelligently) to keep the results in memory  
Additional information can be read in this [stackoverflow issue](https://stackoverflow.com/questions/45941528/how-to-efficiently-send-a-large-numpy-array-to-the-cluster-with-dask-array/45941529#45941529) or see an example in [this post](http://matthewrocklin.com/blog/work/2017/01/12/dask-dataframes)   
This concept should also  be used when running a code within a script (rather then a jupyter notebook) which incoperates loops within the code.


In [40]:
# e.g.
_ddf = dd.read_csv(output_dask_dir/'ddf*.csv')
# do some filter
_ddf = client.persist(_ddf)
# do some computations
_ddf.head(2)

,ID,name,x,y,times
0,970,George,0.367233,0.240946,00:00:00
1,999,Yvonne,0.056227,0.627551,00:00:01


# Group By - custom aggregations
In addition to the [groupby notebook example](https://github.com/dask/dask-examples/blob/main/dataframes/02-groupby.ipynb) that is in the repository -  
This is another example how to try to eliminate the use of `groupby.apply`.   
In this example we are grouping columns into unique lists.

#### Pandas

In [41]:
# prepare pandas dataframe
pdf = pdf.assign(time=pd.to_datetime(pdf.index).time)
pdf['seconds'] = pdf.time.astype(str).str[-2:]
cols_for_demo =['name', 'ID','seconds']
pdf[cols_for_demo].head()

,name,ID,seconds
0,George,970,00
1,Yvonne,999,00
2,Yvonne,962,00
3,Norbert,1019,00
4,Laura,999,00


In [42]:
pdf_gb = pdf.groupby(pdf.name)
gp_col = ['ID', 'seconds']
list_ser_gb = [pdf_gb[att_col_gr].apply
               (lambda x: list(set(x.to_list()))) 
               for att_col_gr in gp_col]

In [43]:
%%time
df_edge_att = pdf_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')      
print(df_edge_att.head(2))

       Weight                                                 ID  \
name                                                               
Alice   99554  [1024, 1025, 1026, 1027, 1028, 1029, 1030, 103...   
Bob     99717  [1024, 1025, 1026, 1027, 1028, 1029, 1030, 103...   

                                                 seconds  
name                                                      
Alice  [33, 72, 74, 55, 38, 30, 59, 42, 16, 51, 96, 9...  
Bob    [33, 72, 74, 55, 38, 30, 59, 42, 16, 51, 96, 9...  
CPU times: user 21.2 ms, sys: 0 ns, total: 21.2 ms
Wall time: 20.5 ms


* Remeber that in any some cases `Pandas` is more efficiante (assuming that you can load all the data into the RAM).  

#### Dask

In [44]:
def set_list_att(x: dd.Series):
        return list(set([item for item in x.values]))
ddf['seconds'] = ddf.times.astype(str).str[-2:]
ddf = client.persist(ddf)
ddf[cols_for_demo].head(2)

,name,ID,seconds
0,George,970,00
1,Yvonne,999,01


In [45]:
ddf.columns

Index(['ID', 'name', 'x', 'y', 'times', 'seconds'], dtype='object')

In [46]:
df_gb = ddf.groupby(ddf.name)
gp_col = ['ID', 'seconds']
list_ser_gb = [df_gb[att_col_gr].apply(set_list_att
                ,meta=pd.Series(dtype='object', name=f'{att_col_gr}_att')) 
               for att_col_gr in gp_col]

In [47]:
%%time
df_edge_att = df_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
    df_edge_att = df_edge_att.join(ser.to_frame(), how='left')
df_edge_att.head(2)

ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   ['name']
  Missing: [0]

We can do better...   
Using [dask custom aggregation](https://docs.dask.org/en/latest/generated/dask.dataframe.groupby.Aggregation.html) is consideribly better

In [48]:
import itertools
custom_agg = dd.Aggregation(
    'custom_agg', 
    lambda s: s.apply(set), 
    lambda s: s.apply(lambda chunks: list(set(itertools.chain.from_iterable(chunks)))),)

In [49]:
%%time
df_gb = ddf.groupby(ddf.name)
gp_col = ['ID', 'seconds']
list_ser_gb = [df_gb[att_col_gr].agg(custom_agg) for att_col_gr in gp_col]
df_edge_att = df_gb.size().to_frame(name="Weight")
for ser in list_ser_gb:
        df_edge_att = df_edge_att.join(ser.to_frame(), how='left')
df_edge_att.head(2)  

CPU times: user 204 ms, sys: 9.71 ms, total: 214 ms
Wall time: 1.3 s


,Weight,ID,seconds
name,,,
Alice,99554,"[1024, 1025, 1026, 1027, 1028, 1029, 1030, 103...","[48, 55, 02, 29, 27, 57, 52, 35, 34, 47, 16, 3..."
Bob,99717,"[1024, 1025, 1026, 1027, 1028, 1029, 1030, 103...","[48, 55, 02, 29, 27, 57, 52, 35, 34, 16, 47, 3..."


## [Debugging](https://docs.dask.org/en/latest/debugging.html)
Debugging may be challenging...
1. Run code without client 
2. Use Dashboard profiler
3. Verify integrity of DAG

### Corrupted DAG  
In this example we show that once the DAG is currupted you may need to reset the calculation

In [50]:
# reset dataframe
ddf = dask.datasets.timeseries()
ddf.head(1)

,id,name,x,y
timestamp,,,,
2000-01-01,1033,Jerry,-0.927602,0.194554


In [51]:
def func_dist2(df, coor_x, coor_y):
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())^2    # `^` <-- wrong syntax
                     +  (df[coor_y] - df[coor_y].shift())^2 )  # `^` <-- wrong syntax
    return dist
ddf['col'] = ddf.map_partitions(func_dist2, coor_x='x', coor_y='y'
                                , meta=('float'))

/usr/share/miniconda3/envs/dask-examples/lib/python3.9/site-packages/dask/dataframe/core.py:6335: FutureWarning: Meta is not valid, `map_partitions` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


Is everything OK?

* Even if the function is corrected the DAG is corrupted

In [52]:
# Still results with an error
def func_dist2(df, coor_x, coor_y):
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2  # `**` <-- correct syntax
                     +  (df[coor_y] - df[coor_y].shift())**2 )  # `**` <-- correct syntax
    return dist
ddf['col'] = ddf.map_partitions(func_dist2, coor_x='x', coor_y='y'
                                , meta=('float'))

/usr/share/miniconda3/envs/dask-examples/lib/python3.9/site-packages/dask/dataframe/core.py:6335: FutureWarning: Meta is not valid, `map_partitions` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


We need to reset the dataframe

In [53]:
ddf = dask.datasets.timeseries()
def func_dist2(df, coor_x, coor_y):
    dist =  np.sqrt ( (df[coor_x] - df[coor_x].shift())**2    #corrected math function
                     +  (df[coor_y] - df[coor_y].shift())**2 )
    return dist
ddf['col'] = ddf.map_partitions(func_dist2, coor_x='x', coor_y='y'
                                , meta=('float'))
ddf.head(2)

/usr/share/miniconda3/envs/dask-examples/lib/python3.9/site-packages/dask/dataframe/core.py:6335: FutureWarning: Meta is not valid, `map_partitions` expects output to be a pandas object. Try passing a pandas object as meta or a dict or tuple representing the (name, dtype) of the columns. In the future the meta you passed will not work.
  warnings.warn(


,id,name,x,y,col
timestamp,,,,,
2000-01-01 00:00:00,1029,Edith,0.421983,-0.617362,NaN
2000-01-01 00:00:01,1015,Zelda,0.970755,0.478438,1.225532
